In [1]:
# 1. Clone the Repository
%cd /content
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting
%cd gaussian-splatting

# 2. Install Requirements (Compiling C++ Kernels...)
print("⏳ Installing dependencies... (This takes ~5-8 minutes)")
!pip install -q plyfile tqdm
!pip install -q submodules/diff-gaussian-rasterization
!pip install -q submodules/simple-knn
print("Installation Complete!")

/content
Cloning into 'gaussian-splatting'...
remote: Enumerating objects: 1053, done.
remote: Total 1053 (delta 0), reused 0 (delta 0), pack-reused 1053 (from 1)
Receiving objects: 100% (1053/1053), 78.70 MiB | 22.42 MiB/s, done.
Resolving deltas: 100% (604/604), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core.git) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization.git) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/fused-ssim' (https://github.com/rahul-goel/fused-ssim.git) registered for path 'submodules/fused-ssim'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/gaussian-splatting/SIBR_viewers'...
remote: Enumerating objects: 3293, done.        
remote: Counting objects: 100% (322/322), done.        
remote: Compressing objects:

In [3]:
import os
import shutil

# Your dataset root
dataset_path = "/content/drive/MyDrive/dataset/custom-dataset"

# 1. Define Source (Where your data is now)
# We use 'colmap_output_1' because that's where you found points3D.bin earlier
source_folder = os.path.join(dataset_path, "colmap_output_1", "0")

# 2. Define Target (Where the official code wants it)
target_folder = os.path.join(dataset_path, "sparse", "0")

# 3. Copy the data
if os.path.exists(source_folder):
    if not os.path.exists(target_folder):
        print(f"Copying data from '{source_folder}' to '{target_folder}'...")
        shutil.copytree(source_folder, target_folder)
        print("Data structure fixed! You are ready to train.")
    else:
        print("'sparse/0' folder already exists. Skipping copy.")
else:
    print(f"Error: Could not find source folder at {source_folder}")
    print("Check your Drive folders manually.")

'sparse/0' folder already exists. Skipping copy.


In [5]:
print("Installing COLMAP... (Takes ~1 min)")
!apt-get update -q
!apt-get install -y colmap -q
print("✅ COLMAP Installed.")

Installing COLMAP... (Takes ~1 min)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,225 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,572 kB]
Get:13 http://archive.ubuntu.com/ub

In [9]:
import os

# --- PATH CONFIGURATION ---
dataset_root = "/content/drive/MyDrive/dataset/custom-dataset"

# 1. The Correct Image Folder (custom_data_1)
images_path = os.path.join(dataset_root, "custom_data_1")

# 2. The Sparse Model (Where points3D.bin is)
input_model = os.path.join(dataset_root, "sparse", "0")

# 3. The Output Folder
output_path = os.path.join(dataset_root, "output_undistorted")

print(f"Reading images from: {images_path}")
print(f"Undistorting to: {output_path}")

# Run COLMAP Undistorter
!colmap image_undistorter \
    --image_path "{images_path}" \
    --input_path "{input_model}" \
    --output_path "{output_path}" \
    --output_type COLMAP \
    --max_image_size 2000

print("\nUndistortion Complete! You can now run the training.")

Reading images from: /content/drive/MyDrive/dataset/custom-dataset/custom_data_1
Undistorting to: /content/drive/MyDrive/dataset/custom-dataset/output_undistorted

Reading reconstruction

 => Reconstruction with 85 images and 86050 points

Image undistortion

Undistorting image [1/85]
Undistorting image [2/85]
Undistorting image [3/85]
Undistorting image [4/85]
Undistorting image [5/85]
Undistorting image [6/85]
Undistorting image [7/85]
Undistorting image [8/85]
Undistorting image [9/85]
Undistorting image [10/85]
Undistorting image [11/85]
Undistorting image [12/85]
Undistorting image [13/85]
Undistorting image [14/85]
Undistorting image [15/85]
Undistorting image [16/85]
Undistorting image [17/85]
Undistorting image [18/85]
Undistorting image [19/85]
Undistorting image [20/85]
Undistorting image [21/85]
Undistorting image [22/85]
Undistorting image [23/85]
Undistorting image [24/85]
Undistorting image [25/85]
Undistorting image [26/85]
Undistorting image [27/85]
Undistorting image [

In [11]:
import os
import shutil

# The path where the files currently are
current_sparse = "/content/drive/MyDrive/dataset/custom-dataset/output_undistorted/sparse"

# The path where they NEED to be
target_zero = os.path.join(current_sparse, "0")

print(f"Checking: {current_sparse}")

# 1. Create the '0' folder
if not os.path.exists(target_zero):
    os.makedirs(target_zero)
    print("Created '0' subfolder.")

# 2. Move the 3 binary files
files = ["cameras.bin", "images.bin", "points3D.bin"]
moved_count = 0

for f in files:
    src = os.path.join(current_sparse, f)
    dst = os.path.join(target_zero, f)

    if os.path.exists(src):
        shutil.move(src, dst)
        print(f"Moved {f} -> sparse/0/")
        moved_count += 1
    elif os.path.exists(dst):
        print(f"{f} is already inside '0'.")
        moved_count += 1
    else:
        print(f"Missing {f}")

if moved_count == 3:
    print("\nAll files fixed! You can run training now.")

Checking: /content/drive/MyDrive/dataset/custom-dataset/output_undistorted/sparse
Created '0' subfolder.
Moved cameras.bin -> sparse/0/
Moved images.bin -> sparse/0/
Moved points3D.bin -> sparse/0/

All files fixed! You can run training now.


In [12]:
!python train.py \
    -s /content/drive/MyDrive/dataset/custom-dataset/output_undistorted \
    -m /content/drive/MyDrive/dataset/custom-dataset/output_model \
    --iterations 7000

2025-12-23 09:37:18.407573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766482638.427302    6882 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766482638.433239    6882 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766482638.448251    6882 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766482638.448274    6882 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766482638.448278    6882 computation_placer.cc:177] computation placer alr